每种语言都有很多的单词组成, 一个单词如何转换称为一个计算机可以处理的向量? 方法可能有很多种, 这里讲述了一种基础的算法: **word2vec**. 这个算法又分成两种: skip-gram和cbow. 算法的思想是什么?　举个例子，看下面两句话：(1) My **mom** love me. (2) My **mother** love me. 我们这里训练一个单隐层的神经网络, 给定一个句子中的中间的一个单词, 预测其他单词出现在这个单词周围的概率(即成对出现的概率).

![jupyter](./images/01_01/training_data.png)
**图1 训练集的构造**

以一个单词为中心, window size设置为2, 上图展示了构造的单词对训练集. 比如第一行的两个样本(the, quick), (the, brown), 当the作为输入的时候, 要求网络输出quick和brown的概率最大. 

1. 收集很多的语料, 即很多段文本, 比如可以取新闻, 报刊上面的文本;
2. 对这些文本进行分词, 英文分词比较简单, 因为天然有空格, 中文分词可以使用jieba这个包来进行分词;
3. 设置window size (比如:2), 以每个单词为中心点, 构造训练集(即单词对, 如上图所示);
4. 将所有的单词收集起来作为一个词典, 假设词典的大小是$N$;
5. 将每个单词进行one-hot编码, 每个单词的向量维度是: $x \in R^N$.
6. 设置神经网络隐层的神经元数量为$M$.
7. 输入层到隐层的要学习的权重矩阵的维度是: $W_1 \in R ^{N \times M}$.
8. 隐层到输出层的要学习的权重矩阵的维度是: $W_2 \in R^{M \times N}$
9. 那3中的训练集对网络进行训练, 我们的目的是得到$W_1$, 由于输入的都是one-hot编码的, 因此网络训练完成之后第$i$个单词对应的词向量就是(行向量): $W_1(i,:)$ (即$W_1$的第i列就是第i个单词的词向量, 依次类推).

![jupyter](./images/01_01/word2vec_weight_matrix_lookup_table.png)
**图2 W权重矩阵示意图**

由于输入是一个one-hot的向量, 如**图3**所示, 第4个单词的词向量就对应权重矩阵$W_1$的第4行, 训练的结果我们把$W1$保存下来就行了, 里面是所有单词的词向量.

![jupyter](./images/01_01/matrix_mult_w_one_hot.png)
*图3 词向量计算*

![jupyter](./images/01_01/skip_gram_net_arch.png)
*图4 word2vec结构*

*图4*给出了word2vec网络的结构图, 这里词典大小是10000, 隐层神经元数量是300, 输入一个ants的one-hot向量, 输出词典中所有其他单词出现的概率值.

![jupyter](./images/01_01/output_weights_function.png)
*图5 输入到输出*

公式推导:

![jupyter](./images/01_01/概率计算.png)
*图6 概率计算*

为了方便, 每个单词的one-hot向量还是设定还是一个列向量, $V$表示词典的大小, 每个单词的one-hot向量的维度是: $ \mathbb{R}^{V\times 1}$, 由于每个单词都是onehot向量, 因此这些向量都是正交的:
$$
(w^{hotel})^Tw^{motal} = (w^{hotel})^T w^{cat} = 0
$$

对于语料中单词的每个位置: $t=1,...,T$, 在一个窗口大小为$m$的范围内预测周围的单词, 给定中心单词$w_t$, 最大化后验概率:
$$
L(\theta) = \prod_{t=1}^T \prod_{-m \leq j \leq m, j\neq 0} P(w_{t+j}|w_t;\theta)
$$
上面这个式子不好求解, 我们转换一下, 目标函数如下:
$$
J(\theta) = -\frac{1}{T} \sum_{t=1}^T \sum_{-m \leq j \leq m, j\neq 0} log(P(w_{t+j}|w_t;\theta))
$$
如何计算$P(w_{t+j}|w_t;\theta)$呢?

对于每个单词$w$, 从$W_1$和$W_2$中分别有一个向量相对应: 如果$w$是中心词, 则向量为$v_w$; 如果$w$是上下文词, 则向量为$u_w$. 则对于中心词$c$和上下文词$o$, 有:
$$
P(o|c) = \frac{exp(u_o^T v_c)}{\sum_{w\in V} exp(u_w^T v_c)}
$$


    Skip-gram是中心词预测上下文单词, CBOW是上下文单词预测中心词.

**参考**:

* [1] CS224n: http://web.stanford.edu/class/cs224n/
* [2] http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/